In [1]:
import warnings

warnings.filterwarnings("ignore")
from pathlib import Path
import pandas as pd
import numpy as np
from tbdynamics.calibration.utils import (
    get_bcm,
    calculate_covid_diff_quantiles,
)
import arviz as az
from tbdynamics.inputs import load_targets
from tbdynamics.constants import  quantiles
from estival.sampling import tools as esamp
import xarray as xr

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
OUT_PATH = Path.cwd() / 'runs/best2208'
idata = az.from_netcdf(OUT_PATH / 'calib_full_out.nc')
burnt_idata = idata.sel(draw=np.s_[50000:])
idata_extract = az.extract(burnt_idata, num_samples=300)

In [25]:
inference_data = az.convert_to_inference_data(idata_extract.reset_index('sample'))

In [26]:
az.to_netcdf(inference_data, 'inference_data.nc')

'inference_data.nc'

In [27]:
loaded_inference_data = az.from_netcdf('inference_data.nc')

In [28]:
loaded_inference_data

Inference data with groups:
	> posterior

In [15]:
extracted_dataset = xr.Dataset(idata_extract)
extracted_idata = az.InferenceData(posterior=extracted_dataset)

In [29]:
params = {
    "start_population_size": 2000000.0,
    "seed_time": 1805.0,
    "seed_num": 1.0,
    "seed_duration": 1.0,
}
covid_effects = {
    'detection_reduction':True,
    'contact_reduction':True
}

In [30]:
quantile_ouputs_diff = calculate_covid_diff_quantiles(params, extracted_idata, ['cumulative_diseased', 'cumulative_deaths'],[2021.0,2022.0],1)


In [31]:
quantile_ouputs_diff

{'abs': {'cumulative_diseased':                 0.025          0.250          0.500          0.750  \
  2021.0 -189770.727594 -112093.666508  -71605.748904  -38839.359199   
  2022.0 -621999.151194 -356742.202775 -229949.855087 -116326.412162   
  
                0.975  
  2021.0 -4234.440777  
  2022.0  4964.960776  ,
  'cumulative_deaths':                0.025         0.250        0.500        0.750         0.975
  2021.0  -4131.709658  -1189.005795   455.921198  1730.089705   4368.406692
  2022.0 -31569.749441 -12584.446690 -2846.145898  4539.747437  20074.655677},
 'rel': {'cumulative_diseased':            0.025     0.250     0.500     0.750     0.975
  2021.0 -0.028439 -0.019804 -0.012543 -0.006744 -0.000795
  2022.0 -0.077138 -0.053356 -0.032557 -0.016208  0.000759,
  'cumulative_deaths':            0.025     0.250     0.500     0.750     0.975
  2021.0 -0.004751 -0.001544  0.000616  0.002547  0.006292
  2022.0 -0.031151 -0.014747 -0.003401  0.005690  0.024896}}